In [80]:
import pandas as pd
import numpy as np

import string  
import nltk
import re
import scipy
import matplotlib.pyplot as plt
import urllib 
import sklearn
from sklearn import preprocessing, metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, f1_score
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
display(train)
display(test)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [81]:
train['keyword'].fillna('no keyword', inplace = True) 
train['keyword'] = train['keyword'].str.replace('%20', ' ')
train['location'].fillna('no location', inplace = True)

test['keyword'].fillna('no keyword', inplace = True) 
test['keyword'] = test['keyword'].str.replace('%20', ' ')
test['location'].fillna('no location', inplace = True)

train['length'] = train['text'].str.len()

test['length'] = test['text'].str.len()

train.head()

,id,keyword,location,text,target,length
0,1,no keyword,no location,Our Deeds are the Reason of this #earthquake M...,1,69
1,4,no keyword,no location,Forest fire near La Ronge Sask. Canada,1,38
2,5,no keyword,no location,All residents asked to 'shelter in place' are ...,1,133
3,6,no keyword,no location,"13,000 people receive #wildfires evacuation or...",1,65
4,7,no keyword,no location,Just got sent this photo from Ruby #Alaska as ...,1,88


In [82]:
#limpiando el texto

import nltk
nltk.download('stopwords') 
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

#FUNCIONES UTILES

def only_letters(tweet):
    tweet = re.sub(r'http\S*', '', tweet)
    tweet = re.sub(r'[^a-z\s]', '', tweet)
    return tweet

stop_words=set(stopwords.words("english"))

def filter_stopwords(tokenized_text):
    filtered_words=[]
    for w in tokenized_text:
        if w not in stop_words:
            filtered_words.append(w)
    return filtered_words

lemmatizer = WordNetLemmatizer()

def lemmatize_tweet(tweet):
    lemmatized_words = []
    for word in tweet:
        lemmatized_words.append(lemmatizer.lemmatize(word))
    return lemmatized_words

def transform_to_text(tweet_words):
    return " ".join(tweet_words)

#Cleaning text

train['clean_text'] = train['text'].str.lower()

train['clean_text'] = train['clean_text'].apply(only_letters)    

test['clean_text'] = test['text'].str.lower()

test['clean_text'] = test['clean_text'].apply(only_letters) 

#Tokenización

train['clean_text'] = train['clean_text'].apply(word_tokenize)

test['clean_text'] = test['clean_text'].apply(word_tokenize)

#Remove stopwords

train['clean_text'] = train['clean_text'].apply(filter_stopwords) 

test['clean_text'] = test['clean_text'].apply(filter_stopwords) 


#Lemmatization                                                                       

train['clean_text'] = train['clean_text'].apply(lemmatize_tweet)  

train['clean_text'] = train['clean_text'].apply(transform_to_text)

test['clean_text'] = test['clean_text'].apply(lemmatize_tweet)  

test['clean_text'] = test['clean_text'].apply(transform_to_text)

display(train)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hijos.ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hijos.ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hijos.ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,id,keyword,location,text,target,length,clean_text
0,1,no keyword,no location,Our Deeds are the Reason of this #earthquake M...,1,69,deed reason earthquake may allah forgive u
1,4,no keyword,no location,Forest fire near La Ronge Sask. Canada,1,38,forest fire near la ronge sask canada
2,5,no keyword,no location,All residents asked to 'shelter in place' are ...,1,133,resident asked shelter place notified officer ...
3,6,no keyword,no location,"13,000 people receive #wildfires evacuation or...",1,65,people receive wildfire evacuation order calif...
4,7,no keyword,no location,Just got sent this photo from Ruby #Alaska as ...,1,88,got sent photo ruby alaska smoke wildfire pour...
...,...,...,...,...,...,...,...
7608,10869,no keyword,no location,Two giant cranes holding a bridge collapse int...,1,83,two giant crane holding bridge collapse nearby...
7609,10870,no keyword,no location,@aria_ahrary @TheTawniest The out of control w...,1,125,ariaahrary thetawniest control wild fire calif...
7610,10871,no keyword,no location,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,65,utckm volcano hawaii
7611,10872,no keyword,no location,Police investigating after an e-bike collided ...,1,137,police investigating ebike collided car little...


In [83]:
#agregando algunos features

keywords_list = list(set(train.keyword))  #todas las keywords únicas

locations_list = list(set(train.location)) #todas las locaciones únicas
locations_list.remove('no location')

def amount_keywords(tweet):
    cont = 0
    for keyword in keywords_list:
        if keyword in tweet:
            cont += 1
    return cont

train['amount_words'] = train['clean_text'].str.split(' ').apply(lambda x: len(x))  

test['amount_words'] = test['clean_text'].str.split(' ').apply(lambda x: len(x))  

#conviene que cuente del tweet limpio o del tweet original??

train['amount_keywords_in_tweet'] = train['clean_text'].apply(lambda x: amount_keywords(x))

test['amount_keywords_in_tweet'] = test['clean_text'].apply(lambda x: amount_keywords(x))


tiene_mas_de_un_kw = train[train['amount_keywords_in_tweet'] > 1]

tiene_un_solo_kw = train[train['amount_keywords_in_tweet'] == 1]

no_tiene_kw = train[train['amount_keywords_in_tweet'] == 0]

porcentaje_target_mas_de_un_kw = tiene_mas_de_un_kw['target'].mean()
print("porcentaje target de tweets con mas de un kw", porcentaje_target_mas_de_un_kw)

porcentaje_target_un_solo_kw = tiene_un_solo_kw['target'].mean()
print("porcentaje target de tweets con un solo kw", porcentaje_target_un_solo_kw)

porcentaje_target_sin_kw = no_tiene_kw['target'].mean()
print("porcentaje target de tweets sin kw", porcentaje_target_sin_kw)

print("el maximo de cant de keywords en un tweet es", tiene_mas_de_un_kw['amount_keywords_in_tweet'].max())

def amount_locations(tweet):    #esta función podría unificarse con la de keywords pasandole por parámetro la lista a recorrer
    cont = 0
    for location in locations_list:
        if location in tweet:
            cont += 1
    return cont

train['amount_locations_in_tweet'] = train['clean_text'].apply(lambda x: amount_locations(x))

test['amount_locations_in_tweet'] = test['clean_text'].apply(lambda x: amount_locations(x))


mas_de_1_location = train[train['amount_locations_in_tweet'] > 1]

un_solo_location = train[train['amount_locations_in_tweet'] == 1]

no_tiene_location = train[train['amount_locations_in_tweet'] == 0]

porcentaje_target_mas_de_un_location = mas_de_1_location['target'].mean()
print("porcentaje target de tweets con mas de una locacion", porcentaje_target_mas_de_un_location)

porcentaje_target_un_solo_location = un_solo_location['target'].mean()
print("porcentaje target de tweets con una sola locacion", porcentaje_target_un_solo_location)

porcentaje_target_sin_location = no_tiene_location['target'].mean()
print("porcentaje target de tweets sin locacion", porcentaje_target_sin_location)

print("el maximo de cant de locaciones en un tweet es", mas_de_1_location['amount_locations_in_tweet'].max())

porcentaje target de tweets con mas de un kw 0.606816584680253
porcentaje target de tweets con un solo kw 0.3446327683615819
porcentaje target de tweets sin kw 0.2025862068965517
el maximo de cant de keywords en un tweet es 7
porcentaje target de tweets con mas de una locacion 0.504933504933505
porcentaje target de tweets con una sola locacion 0.4489409984871407
porcentaje target de tweets sin locacion 0.34382107657316147
el maximo de cant de locaciones en un tweet es 8


In [84]:
mas_de_un_location_y_mas_de_un_keyword = train[(train['amount_locations_in_tweet'] > 1) & (train['amount_keywords_in_tweet'] > 1)]
porcentaje_target_mas_de_un_location_y_mas_de_un_keyword = mas_de_un_location_y_mas_de_un_keyword.target.mean()
porcentaje_target_mas_de_un_location_y_mas_de_un_keyword

0.6990196078431372

In [85]:
#más features
def get_hashtags(s):
    return list(part[1:] for part in s.split() if part.startswith('#'))

train['hashtags'] = train['text'].apply(get_hashtags)
test['hashtags'] = test['text'].apply(get_hashtags)

def hashtag_length_proportion(hashtags, length):
    return len(''.join(hashtags))/length

#VER SI DEJAR ESTEEE
train['len_hash_over_text'] = train.apply(lambda data: hashtag_length_proportion(data['hashtags'], data['length']), axis=1)
test['len_hash_over_text'] = test.apply(lambda data: hashtag_length_proportion(data['hashtags'], data['length']), axis=1)

def get_mentioned_users(s):
    return list(part[1:] for part in s.split() if part.startswith('@'))

train['users'] = train['text'].apply(get_mentioned_users)
test['users'] = test['text'].apply(get_mentioned_users)

def amount_hashtags(l):
    return len(l)

train['amount_hashtags'] = train['hashtags'].apply(amount_hashtags)
test['amount_hashtags'] = test['hashtags'].apply(amount_hashtags)

def amount_users(l):
    return len(l)

def has_url(text):
    return int('http' in text)

train['has_url'] = train['text'].apply(has_url)
test['has_url'] = test['text'].apply(has_url)

def find_url(text):
    urls = re.findall(r'(https?://\S+)', text)
    return urls

train['urls'] = train['text'].apply(find_url)
test['urls'] = test['text'].apply(find_url)
train['amount_urls'] = train['urls'].apply(lambda x: len(x))
test['amount_urls'] = test['urls'].apply(lambda x: len(x))

train['amount_users'] = train['users'].apply(amount_users)
test['amount_users'] = test['users'].apply(amount_users)


In [86]:
#construyendo los datos de entrenamiento y de test
selected_df = train[['length','amount_words','amount_keywords_in_tweet','amount_locations_in_tweet', 'amount_urls','amount_users', 'amount_hashtags','len_hash_over_text', 'has_url']] 
#ver después de codificar las variables categóricas para agregarlas
X, y = selected_df,train['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [89]:
X_train

,length,amount_words,amount_keywords_in_tweet,amount_locations_in_tweet,amount_urls,amount_users,amount_hashtags,len_hash_over_text,has_url
4996,140,12,2,2,1,0,1,0.078571,1
3263,98,11,1,1,0,2,1,0.122449,0
4907,143,14,2,1,1,1,0,0.000000,1
2855,118,9,1,1,1,0,0,0.000000,1
4716,82,9,2,4,0,3,1,0.158537,0
...,...,...,...,...,...,...,...,...,...
5226,96,8,1,1,0,1,0,0.000000,0
5390,132,15,2,0,0,0,0,0.000000,0
860,121,10,1,1,2,0,0,0.000000,1
7603,136,11,1,2,1,0,0,0.000000,1


**Random Forest como problema de regresión**

In [119]:
#Random forest por defecto
rf_model = RandomForestRegressor(random_state=1)
rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))
rf_model.feature_importances_

RMSE: 0.473488


array([0.348135  , 0.1691422 , 0.12285792, 0.10963829, 0.04618134,
       0.05828546, 0.02782684, 0.0885487 , 0.02938426])

In [120]:
#Random forest prueba2
rf_model = RandomForestRegressor(random_state = 1, n_estimators = 100, max_features = 4, min_samples_split = 3, max_depth = 3)

rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.451396


In [122]:
#Random forest prueba3
rf_model = RandomForestRegressor(random_state = 1, n_estimators = 200, max_features = 2, min_samples_split = 5, max_depth = 6,
                                min_samples_leaf = 4)

rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.445672


In [123]:
#Random forest prueba4
rf_model = RandomForestRegressor(random_state = 1, n_estimators = 200, max_features = 6, min_samples_split = 8, max_depth = 6,
                                min_samples_leaf = 10)

rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.443386


In [128]:
#Random forest prueba5
rf_model = RandomForestRegressor(random_state = 1, n_estimators = 80, max_features = 7, min_samples_split = 10, max_depth = 7,
                                min_samples_leaf = 25)

rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.441838


In [129]:
#Random forest prueba6
rf_model = RandomForestRegressor(random_state = 1, n_estimators = 80, max_features = 7, min_samples_split = 10, max_depth = 9,
                                min_samples_leaf = 25)

rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.440774


In [131]:
#Random forest prueba7
rf_model = RandomForestRegressor(random_state = 1, n_estimators = 85, max_features = 7, min_samples_split = 10, max_depth = 9,
                                min_samples_leaf = 25)
#VARIANDO MIN_SAMPLES_SPLIT NO CAMBIA NADA
rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.440740


In [136]:
#Random forest prueba8
rf_model = RandomForestRegressor(random_state = 1, n_estimators = 85, max_features = 7, min_samples_split = 10, max_depth = 9,
                                min_samples_leaf = 11)
rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.440364


In [144]:
#Random forest prueba9
rf_model = RandomForestRegressor(random_state = 1, n_estimators = 85, max_features = 6, min_samples_split = 10, max_depth = 12,
                                min_samples_leaf = 11)

rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.440225


In [145]:
#Random forest prueba10
rf_model = RandomForestRegressor(random_state = 1, n_estimators = 85, max_features = 5, min_samples_split = 10, max_depth = 12,
                                min_samples_leaf = 11)

rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.439360


In [147]:
#Random forest prueba11
rf_model = RandomForestRegressor(random_state = 1, n_estimators = 85, max_features = 5, min_samples_split = 10, max_depth = 11,
                                min_samples_leaf = 10)

rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.439130


**Random Forest como problema de clasificación**

In [148]:
#Random forest por defecto
rf_model = RandomForestClassifier(random_state=1)
rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

f1score = f1_score(y_test, preds)
print(f'Counts model score: {f1score*100}%')

rf_model.feature_importances_

RMSE: 0.571253
Counts model score: 60.335195530726274%


array([0.39030688, 0.16775926, 0.12297028, 0.08711516, 0.04516936,
       0.04942861, 0.03150647, 0.07937009, 0.02637389])

In [149]:
#Random forest prueba2
rf_model = RandomForestClassifier(random_state = 1, n_estimators = 100, max_features = 4, min_samples_split = 3, max_depth = 3)

rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

f1score = f1_score(y_test, preds)
print(f'Counts model score: {f1score*100}%')

RMSE: 0.558466
Counts model score: 53.20197044334976%


In [150]:
#Random forest prueba3
rf_model = RandomForestClassifier(random_state = 1, n_estimators = 90, max_features = 6, min_samples_split = 3, max_depth = 4,
                                  min_samples_leaf = 2)

rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

f1score = f1_score(y_test, preds)
print(f'Counts model score: {f1score*100}%')

RMSE: 0.557289
Counts model score: 54.38765670202507%


In [151]:
#Random forest prueba4
rf_model = RandomForestClassifier(random_state = 1, n_estimators = 90, max_features = 7, min_samples_split = 8, max_depth = 5,
                                  min_samples_leaf = 15)

rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

f1score = f1_score(y_test, preds)
print(f'Counts model score: {f1score*100}%')

RMSE: 0.556110
Counts model score: 54.75504322766571%


In [154]:
#Random forest prueba5
rf_model = RandomForestClassifier(random_state = 1, n_estimators = 90, max_features = 7, min_samples_split = 10, max_depth = 8,
                                min_samples_leaf = 15)

rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

f1score = f1_score(y_test, preds)
print(f'Counts model score: {f1score*100}%')

RMSE: 0.542967
Counts model score: 61.72208013640239%


In [155]:
#Random forest prueba6
rf_model = RandomForestClassifier(random_state = 1, n_estimators = 90, max_features = 7, min_samples_split = 10, max_depth = 10,
                                min_samples_leaf = 15)

rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

f1score = f1_score(y_test, preds)
print(f'Counts model score: {f1score*100}%')

RMSE: 0.532589
Counts model score: 62.88659793814433%


In [159]:
#Random forest prueba7
rf_model = RandomForestClassifier(random_state = 1, n_estimators = 85, max_features = 7, min_samples_split = 10, max_depth = 10,
                                min_samples_leaf = 15)

rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

f1score = f1_score(y_test, preds)
print(f'Counts model score: {f1score*100}%')

RMSE: 0.533205
Counts model score: 62.89631533847473%


**Probando Randomized Search CV**

In [110]:
params = {
 "max_depth"        : [3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
 "n_estimators"     : [80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 250],
 "min_samples_split": [4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 25],
 "min_samples_leaf" : [4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32],
 "max_features"     : [2, 3, 4, 5, 6, 7, 8, 9]
} 

model = RandomForestClassifier(random_state=1)

random_search = RandomizedSearchCV(model,param_distributions=params,scoring='f1',cv=5,verbose=3,n_iter=100)
random_search.fit(X, y)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] n_estimators=90, min_samples_split=14, min_samples_leaf=10, max_features=6, max_depth=3 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  n_estimators=90, min_samples_split=14, min_samples_leaf=10, max_features=6, max_depth=3, score=0.512, total=   0.8s
[CV] n_estimators=90, min_samples_split=14, min_samples_leaf=10, max_features=6, max_depth=3 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.7s remaining:    0.0s


[CV]  n_estimators=90, min_samples_split=14, min_samples_leaf=10, max_features=6, max_depth=3, score=0.478, total=   0.8s
[CV] n_estimators=90, min_samples_split=14, min_samples_leaf=10, max_features=6, max_depth=3 


...

[CV]  n_estimators=80, min_samples_split=20, min_samples_leaf=20, max_features=5, max_depth=4, score=0.510, total=   0.5s
[CV] n_estimators=80, min_samples_split=20, min_samples_leaf=20, max_features=5, max_depth=4 
[CV]  n_estimators=80, min_samples_split=20, min_samples_leaf=20, max_features=5, max_depth=4, score=0.473, total=   0.5s
[CV] n_estimators=80, min_samples_split=20, min_samples_leaf=20, max_features=5, max_depth=4 
[CV]  n_estimators=80, min_samples_split=20, min_samples_leaf=20, max_features=5, max_depth=4, score=0.551, total=   0.5s
[CV] n_estimators=80, min_samples_split=20, min_samples_leaf=20, max_features=5, max_depth=4 
[CV]  n_estimators=80, min_samples_split=20, min_samples_leaf=20, max_features=5, max_depth=4, score=0.495, total=   0.5s
[CV] n_estimators=80, min_samples_split=20, min_samples_leaf=20, max_features=5, max_depth=4 
[CV]  n_estimators=80, min_samples_split=20, min_samples_leaf=20, max_features=5, max_depth=4, score=0.600, total=   0.5s
[CV] n_estimat

[CV]  n_estimators=170, min_samples_split=24, min_samples_leaf=30, max_features=4, max_depth=12, score=0.586, total=   1.3s
[CV] n_estimators=170, min_samples_split=24, min_samples_leaf=30, max_features=4, max_depth=12 
[CV]  n_estimators=170, min_samples_split=24, min_samples_leaf=30, max_features=4, max_depth=12, score=0.629, total=   1.2s
[CV] n_estimators=140, min_samples_split=6, min_samples_leaf=22, max_features=9, max_depth=11 
[CV]  n_estimators=140, min_samples_split=6, min_samples_leaf=22, max_features=9, max_depth=11, score=0.575, total=   1.6s
[CV] n_estimators=140, min_samples_split=6, min_samples_leaf=22, max_features=9, max_depth=11 
[CV]  n_estimators=140, min_samples_split=6, min_samples_leaf=22, max_features=9, max_depth=11, score=0.544, total=   1.6s
[CV] n_estimators=140, min_samples_split=6, min_samples_leaf=22, max_features=9, max_depth=11 
[CV]  n_estimators=140, min_samples_split=6, min_samples_leaf=22, max_features=9, max_depth=11, score=0.588, total=   1.6s
[C

[CV]  n_estimators=150, min_samples_split=25, min_samples_leaf=20, max_features=8, max_depth=6, score=0.511, total=   1.3s
[CV] n_estimators=150, min_samples_split=25, min_samples_leaf=20, max_features=8, max_depth=6 
[CV]  n_estimators=150, min_samples_split=25, min_samples_leaf=20, max_features=8, max_depth=6, score=0.601, total=   1.4s
[CV] n_estimators=150, min_samples_split=25, min_samples_leaf=20, max_features=8, max_depth=6 
[CV]  n_estimators=150, min_samples_split=25, min_samples_leaf=20, max_features=8, max_depth=6, score=0.580, total=   1.3s
[CV] n_estimators=150, min_samples_split=25, min_samples_leaf=20, max_features=8, max_depth=6 
[CV]  n_estimators=150, min_samples_split=25, min_samples_leaf=20, max_features=8, max_depth=6, score=0.631, total=   1.3s


[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed: 11.5min finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [111]:
random_search.best_estimator_

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=12, max_features=5,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=80,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [112]:
random_search.best_params_

{'n_estimators': 80,
 'min_samples_split': 10,
 'min_samples_leaf': 10,
 'max_features': 5,
 'max_depth': 12}

In [113]:
model = random_search.best_estimator_

#Predicciones set de train
preds = model.predict(X)

f1score = f1_score(train['target'], preds)
print(f'Counts model score: {f1score*100}%')

rmse = np.sqrt(mean_squared_error(train['target'], preds))
print(f'RMSE: %f' % (rmse))

Counts model score: 66.87370600414077%
RMSE: 0.502195


In [114]:
params = {
 "max_depth"        : [ 9, 10, 11, 12, 13, 14, 15],
 "n_estimators"     : [ 74, 76, 78, 80, 82, 84, 86, 88, 90],
 "min_samples_split": [ 7, 8, 9, 10, 11, 12, 13, 14, 15],
 "min_samples_leaf" : [ 7, 8, 9, 10, 11, 12, 13, 14, 15],
 "max_features"     : [ 2, 3, 4, 5, 6, 7, 8, 9]
} 

model = RandomForestClassifier(random_state=1)

random_search = RandomizedSearchCV(model,param_distributions=params,scoring='f1',cv=5,verbose=3,n_iter=100)
random_search.fit(X, y)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] n_estimators=80, min_samples_split=13, min_samples_leaf=15, max_features=4, max_depth=9 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  n_estimators=80, min_samples_split=13, min_samples_leaf=15, max_features=4, max_depth=9, score=0.567, total=   1.2s
[CV] n_estimators=80, min_samples_split=13, min_samples_leaf=15, max_features=4, max_depth=9 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.1s remaining:    0.0s


[CV]  n_estimators=80, min_samples_split=13, min_samples_leaf=15, max_features=4, max_depth=9, score=0.538, total=   1.1s
[CV] n_estimators=80, min_samples_split=13, min_samples_leaf=15, max_features=4, max_depth=9 


...

[CV]  n_estimators=86, min_samples_split=12, min_samples_leaf=12, max_features=4, max_depth=9, score=0.566, total=   0.8s
[CV] n_estimators=86, min_samples_split=12, min_samples_leaf=12, max_features=4, max_depth=9 
[CV]  n_estimators=86, min_samples_split=12, min_samples_leaf=12, max_features=4, max_depth=9, score=0.553, total=   0.8s
[CV] n_estimators=86, min_samples_split=12, min_samples_leaf=12, max_features=4, max_depth=9 
[CV]  n_estimators=86, min_samples_split=12, min_samples_leaf=12, max_features=4, max_depth=9, score=0.599, total=   0.8s
[CV] n_estimators=86, min_samples_split=12, min_samples_leaf=12, max_features=4, max_depth=9 
[CV]  n_estimators=86, min_samples_split=12, min_samples_leaf=12, max_features=4, max_depth=9, score=0.581, total=   0.8s
[CV] n_estimators=86, min_samples_split=12, min_samples_leaf=12, max_features=4, max_depth=9 
[CV]  n_estimators=86, min_samples_split=12, min_samples_leaf=12, max_features=4, max_depth=9, score=0.644, total=   0.8s
[CV] n_estimat

[CV]  n_estimators=78, min_samples_split=10, min_samples_leaf=7, max_features=4, max_depth=12, score=0.591, total=   0.8s
[CV] n_estimators=78, min_samples_split=10, min_samples_leaf=7, max_features=4, max_depth=12 
[CV]  n_estimators=78, min_samples_split=10, min_samples_leaf=7, max_features=4, max_depth=12, score=0.655, total=   0.8s
[CV] n_estimators=82, min_samples_split=10, min_samples_leaf=12, max_features=6, max_depth=12 
[CV]  n_estimators=82, min_samples_split=10, min_samples_leaf=12, max_features=6, max_depth=12, score=0.584, total=   1.0s
[CV] n_estimators=82, min_samples_split=10, min_samples_leaf=12, max_features=6, max_depth=12 
[CV]  n_estimators=82, min_samples_split=10, min_samples_leaf=12, max_features=6, max_depth=12, score=0.565, total=   1.1s
[CV] n_estimators=82, min_samples_split=10, min_samples_leaf=12, max_features=6, max_depth=12 
[CV]  n_estimators=82, min_samples_split=10, min_samples_leaf=12, max_features=6, max_depth=12, score=0.591, total=   1.2s
[CV] n_e

[CV]  n_estimators=80, min_samples_split=13, min_samples_leaf=15, max_features=8, max_depth=11, score=0.564, total=   1.2s
[CV] n_estimators=80, min_samples_split=13, min_samples_leaf=15, max_features=8, max_depth=11 
[CV]  n_estimators=80, min_samples_split=13, min_samples_leaf=15, max_features=8, max_depth=11, score=0.584, total=   1.1s
[CV] n_estimators=80, min_samples_split=13, min_samples_leaf=15, max_features=8, max_depth=11 
[CV]  n_estimators=80, min_samples_split=13, min_samples_leaf=15, max_features=8, max_depth=11, score=0.587, total=   1.2s
[CV] n_estimators=80, min_samples_split=13, min_samples_leaf=15, max_features=8, max_depth=11 
[CV]  n_estimators=80, min_samples_split=13, min_samples_leaf=15, max_features=8, max_depth=11, score=0.644, total=   1.3s


[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:  8.0min finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [115]:
random_search.best_estimator_

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=14, max_features=4,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=13, min_samples_split=7,
                       min_weight_fraction_leaf=0.0, n_estimators=82,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [116]:
random_search.best_params_

{'n_estimators': 82,
 'min_samples_split': 7,
 'min_samples_leaf': 13,
 'max_features': 4,
 'max_depth': 14}

In [117]:
model = random_search.best_estimator_

#Predicciones set de train
preds = model.predict(X)

f1score = f1_score(train['target'], preds)
print(f'Counts model score: {f1score*100}%')

rmse = np.sqrt(mean_squared_error(train['target'], preds))
print(f'RMSE: %f' % (rmse))
#NO MEJORÓ EL RESULTADO CON RESPECTO AL ANTERIOR RANDOM SEARCH

Counts model score: 65.117903930131%
RMSE: 0.512166
